* Steam tag
    * 액션 
    * 싱글 플레이어
    * 인디
    * 어드벤처
    * 시뮬레이션
    * 전략
    * 롤플레잉
    * 캐주얼
    * 멀티플레이어
    * 분위기 있는
    * 2D
    * 풍부한 스토리
    * 웅장한 사운드트랙
    * 협동
    * 오픈월드
    * 폭력
---------------------------------------------------------

In [2]:
from selenium import webdriver
import time
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
class Steam:
    def __init__(self, game_id, cond):
        self.game_id = game_id  # steam game code
        self.cond = cond  # positive or negative only

    def get_review(self):
        browser = webdriver.Chrome(
            "C:\\Users\\korea\\OneDrive\\바탕 화면\\캡스톤디자인 프로젝트\\chromedriver_win32\\chromedriver.exe")
        browser.maximize_window()

        # 페이지 이동
        url = "https://steamcommunity.com/app/" + self.game_id + "/" + \
            self.cond + "reviews/?browsefilter=toprated&p=1&filterLanguage=koreana"
        browser.get(url)

        interval = 2  # 2초에 한번씩 스크롤 다운

        # 현재 문서 높이 가져와서 저장
        prev_height = browser.execute_script(
            "return document.body.scrollHeight")

        # 반복수행
        while True:
            # 스크롤을 가장 아래로 내림
            browser.execute_script(
                "window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
            time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
            curr_height = browser.execute_script(
                "return document.body.scrollHeight")
            if curr_height == prev_height:
                break

            prev_height = curr_height
        print("스크롤 완료")

        soup = BeautifulSoup(browser.page_source, "lxml")

        # 리뷰, 플레이타임, 리뷰추천수
        review_box = soup.find_all(
            "div", attrs={"class": "apphub_CardContentMain"})
        print(len(review_box))

        df_review = []
        df_playtime = []
        df_helpful = []
        df_funny = []
        df_game_cnt = []
        df_date_year = []
        df_date_month = []

        year = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
                '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
        month = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
                 'August', 'September', 'October', 'November', 'December']

        for box in review_box:
            review = box.find(
                "div", attrs={"class": "apphub_CardTextContent"}).get_text().strip()
            review = review.replace('Early Access Review', '')
            review = review.replace('Posted: ', '')
            review = review.split()
            if len(review) == 2:
                pass
            else:
                if (review[2] in year):
                    review = ' '.join(review[3:])
                else:
                    review = ' '.join(review[2:])
            df_review.append(review)

            playtime = box.find("div", attrs={"class": "hours"}).get_text()
            playtime = playtime.split()
            temp1 = []
            for w in playtime:
                w = re.sub(r'[^0-9.]', '', w)
                w = "".join([str(_) for _ in w])
                temp1.append(w)
            temp1 = list(filter(None, temp1))
            df_playtime.append(float(temp1[0]))

            recommend = box.find(
                "div", attrs={"class": "found_helpful"}).get_text().strip()
            recommend = recommend.split()
            temp2 = []  # 유용한 평가, 재미있는 평가, 받은 이모티콘 수
            for w in recommend:
                w = re.findall(r'[0-9]+', w)
                w = "".join([str(_) for _ in w])
                temp2.append(w)
            temp2 = list(filter(None, temp2))

            if len(temp2) == 3:
                df_helpful.append(int(temp2[0]))
                df_funny.append(int(temp2[1]))
            elif len(temp2) == 2:
                df_helpful.append(int(temp2[0]))
                df_funny.append(int(temp2[1]))
            elif len(temp2) == 1:
                df_helpful.append(int(temp2[0]))
                df_funny.append(int(0))
            else:
                df_helpful.append(int(0))
                df_funny.append(int(0))

            # 리뷰 게시 일시
            date = box.find("div", attrs={"class": "date_posted"}).get_text()
            date = date.replace('Posted: ', '')
            date = date.replace(',', '')
            date = date.split()
            
            get_i = 0
            for i in range(len(date)):  # 년도 추출
                if date[i] in year:
                    get_i = i
            if get_i:
                df_date_year.append(date[get_i])
            else:
                df_date_year.append(2021)
            
            get_i = 0
            for i in range(len(date)):  # 월 추출
                if date[i] in month:
                    df_date_month.append(date[i])

        # 구매한 게임 수
        games = soup.find_all(
            "div", attrs={"class": "apphub_CardContentAuthorBlock tall"})

        for game in games:
            game_cnt = game.find(
                "div", attrs={"class": "apphub_CardContentMoreLink ellipsis"}).get_text()
            game_cnt = re.sub(r'[^0-9]', '', game_cnt)
            df_game_cnt.append(game_cnt)
            
        print(len(df_review))
        print(len(df_playtime))
        print(len(df_helpful))
        print(len(df_funny))
        print(len(df_game_cnt))
        print(len(df_date_year))
        print(len(df_date_month))

        final_data = {'리뷰': df_review, "플레이타임": df_playtime,
                      '유용한 평가': df_helpful, '재밌는 평가': df_funny, 
                      '구매게임 수': df_game_cnt, '작성 년도': df_date_year, '작성 월': df_date_month}
        df = pd.DataFrame(final_data)
        
        df['작성 월'] = df['작성 월'].map({'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7,
                 'August':8, 'September':9, 'October':10, 'November':11, 'December':12})

        # positive : 1  / negative : 0
        if self.cond == 'positive':
            df['label'] = 1
        else:
            df['label'] = 0
        return df

# Stardew Valley 긍정 리뷰 데이터

In [4]:
g1_pos = Steam('413150', 'positive')
g1_r_pos = g1_pos.get_review()

스크롤 완료
550
550
550
550
550
550
550
550


In [5]:
g1_r_pos

,리뷰,플레이타임,유용한 평가,재밌는 평가,구매게임 수,작성 년도,작성 월,label
0,자유로운 플레이를 하고싶은 분들다양한 컨텐츠를 즐기고 좋아하시는 분들발랄하고 기분이...,83.9,849,57,180,2017,6,1
1,너무재밌는데..는데...는데.........한국 유저 많은거 같은데 한글패치는 안나...,392.2,289,6,42,2018,7,1
2,19년1월25일 기준 완벽 한글화!!스타듀벨리(마우스우클릭)->속성->베타 (참여하...,42.0,291,7,80,2019,1,1
3,힐링하러 왔다가 과로로 쓰러지는 게임,104.4,168,73,8,2020,12,1
4,촌구석연애시뮬레이션,106.6,201,43,158,2017,5,1
...,...,...,...,...,...,...,...,...
545,조금 해봤는데 재밌는 것 같네요,415.1,4,1,20,2018,5,1
546,이 게임 하나 사면 몇일은 그냥 보냄 ㄹㅇ,187.4,3,1,54,2018,5,1
547,.,64.1,3,0,89,2018,5,1
548,갓겜,88.2,2,0,244,2017,11,1


# Stardew Valley 부정 리뷰 데이터

In [7]:
g1_neg = Steam('413150', 'negative')
g1_r_neg = g1_neg.get_review()

스크롤 완료
80
80
80
80
80
80
80
80


In [8]:
g1_r_neg

,리뷰,플레이타임,유용한 평가,재밌는 평가,구매게임 수,작성 년도,작성 월,label
0,힐링없는 농촌게임 이 게임으로 진정 여유로운 힐링을 바라는 유저들은 구매 전에 다시...,19.3,36,1,70,2018,2,0
1,나하고 춤 아무도 안춰줌,105.6,19,48,27,2019,9,0
2,나같은 아싸가 하기엔 너무 사람을 만나야함... 난 조용히 농사를 짓고싶어,18.1,13,14,25,2018,6,0
3,무슨짓을 해도 실행이 안됨..,0.9,13,0,2,2019,5,0
4,업데이트해서 재밌는거 많이 생겼지만한국인은 역시 노가다 한다고 심신이 지침하다가 졸...,207.4,4,0,35,2020,12,0
...,...,...,...,...,...,...,...,...
75,환불하고싶어요,1.8,1,1,34,2021,3,0
76,정말정말 재미업슴니다. 하루에 10시간씩 밖에 못하겠어요.,81.8,1,3,12,2021,2,0
77,제 적성에는 별로 안맞았던것같네요 환불어떻게하나요,1.6,0,0,2,2020,8,0
78,인사하는거에서 막혀서 안하게됨 ...,3.1,1,0,9,2020,7,0


# Steam 리뷰 데이터 긍정 부정 통합

In [9]:
new_df = pd.concat([g1_r_pos, g1_r_neg], ignore_index=True)
new_df

,리뷰,플레이타임,유용한 평가,재밌는 평가,구매게임 수,작성 년도,작성 월,label
0,자유로운 플레이를 하고싶은 분들다양한 컨텐츠를 즐기고 좋아하시는 분들발랄하고 기분이...,83.9,849,57,180,2017,6,1
1,너무재밌는데..는데...는데.........한국 유저 많은거 같은데 한글패치는 안나...,392.2,289,6,42,2018,7,1
2,19년1월25일 기준 완벽 한글화!!스타듀벨리(마우스우클릭)->속성->베타 (참여하...,42.0,291,7,80,2019,1,1
3,힐링하러 왔다가 과로로 쓰러지는 게임,104.4,168,73,8,2020,12,1
4,촌구석연애시뮬레이션,106.6,201,43,158,2017,5,1
...,...,...,...,...,...,...,...,...
625,환불하고싶어요,1.8,1,1,34,2021,3,0
626,정말정말 재미업슴니다. 하루에 10시간씩 밖에 못하겠어요.,81.8,1,3,12,2021,2,0
627,제 적성에는 별로 안맞았던것같네요 환불어떻게하나요,1.6,0,0,2,2020,8,0
628,인사하는거에서 막혀서 안하게됨 ...,3.1,1,0,9,2020,7,0


# 빈도수 분석을 위해 리뷰 피쳐만 출력

In [27]:
pos_review = g1_r_pos['리뷰']

In [28]:
pos_review.head()

0    게임시작전 가장 중요한 정보를 알려드리겠습니다.1. 아래사진 처럼 게임버젼을 41버...
1    200시간 넘게 멀드로에서 시작하는 Really CD DA 챌린지 등을 통해 썩을 ...
2                                       남이 하면 재밌어 보이는겜
3    최근들어 게임이 흥하는건 개발자 때문이 절대 아니다그들은 일하지 않는다게임에 애정을...
4                                 2020년에도 얼리엑세스 예상해봅니다
Name: 리뷰, dtype: object

In [29]:
neg_review = g1_r_neg['리뷰']

In [30]:
neg_review.head()

0    굉장히 재미있는 게임은 맞는데 최근 좀비 리스폰 방식 변경 이후 난이도가 안드로메다...
1    사운드 피드백 안준다고 사운드 조절옵션을 제거해버렸고심지어 패치된 사운드중에 누락된...
2                           출시가 2013년인데 아직까지도 얼리얼엑세스야.
3                      풍형이 할땐 재밌었는데...내가 하니까 노잼이야...ㅜㅜ
4    3년이 지났지만 아직 앞서해보는게임이다.3년이 더 지나도 앞서해보는게임 일것이고 질...
Name: 리뷰, dtype: object

# 리뷰 데이터 txt 저장

In [31]:
neg_review.to_csv('neg_review.txt', sep='\t', index=False)
pos_review.to_csv('pos_review.txt', sep='\t', index=False)

In [11]:
# 통합 데이터 저장
new_df.to_csv('stardew_review.txt', sep='\t', index=False)

# data shuffle

In [88]:
df_shuffle = new_df.sample(frac=1).reset_index(drop=True)
df_shuffle

,리뷰,플레이타임,유용한 평가,재밌는 평가,구매게임 수,label
0,이 게임의 얼리엑세스는 끝나지 않는다,123.0,8,0,699,0
1,"2013년 ""앞서 해보는 게임""2014년 ""앞서 해보는 게임""2015년 ""앞서 해보...",77.4,45,17,28,0
2,200시간 넘게 멀드로에서 시작하는 Really CD DA 챌린지 등을 통해 썩을 ...,400.4,48,7,77,1
3,NPC는 언제?????????????,58.3,20,2,412,1
4,ㅋㅋ 상승세인 게임을 스스로 말아먹는 좆병신 개발진이 있다!?!?!?,145.0,13,1,130,0
5,이겜은 표백제에 불행-99가 아니라 행복+99 이런거 붙여놔야함,18.2,61,31,129,1
6,2015년에 사서 조금 플레이하다가 좆병신게임이라고 때려쳤는데 2020년에 중갤 연...,45.8,47,28,56,1
7,ㅇㄴ 장난치냐?? 개발자 니들이 정식 멀티를 못내니까 유저들이 멀티를 만들어서 하던...,50.5,18,0,13,0
8,게임시작전 가장 중요한 정보를 알려드리겠습니다.1. 아래사진 처럼 게임버젼을 41버...,2.7,594,3,192,1
9,굉장히 재미있는 게임은 맞는데 최근 좀비 리스폰 방식 변경 이후 난이도가 안드로메다...,265.8,64,13,73,0
